In [1]:
import os, time, itertools
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
def lrelu(x, th=0.2):
    return tf.maximum(th * x, x)

In [3]:
def Generator(x,isTrain=True,reuse=False):
    with tf.variable_scope('gen', reuse=reuse):
        conv1 = tf.layers.conv2d_transpose(x, 1024, [4, 4], strides=(1, 1), padding='valid')
        batch_norm=tf.layers.batch_normalization(conv1, training=isTrain)
        lrelu1=lrelu(batch_norm,0.2)
        conv2 = tf.layers.conv2d_transpose(lrelu1, 512, [4, 4], strides=(2, 2), padding='same')
        lrelu2 = lrelu(tf.layers.batch_normalization(conv2, training=isTrain), 0.2)
        conv5 = tf.layers.conv2d_transpose(lrelu2, 1, [4, 4], strides=(2, 2), padding='same')
        o = tf.nn.tanh(conv5)
        return o
        

In [4]:
def discriminator(x, isTrain=True, reuse=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        # 1st hidden layer
        conv1 = tf.layers.conv2d(x, 128, [4, 4], strides=(2, 2), padding='same')
        lrelu1 = lrelu(conv1, 0.2)

        # 2nd hidden layer
        conv2 = tf.layers.conv2d(lrelu1, 256, [4, 4], strides=(2, 2), padding='same')
        lrelu2 = lrelu(tf.layers.batch_normalization(conv2, training=isTrain), 0.2)
        conv5 = tf.layers.conv2d(lrelu2, 1, [4, 4], strides=(1, 1), padding='valid')
        o = tf.nn.sigmoid(conv5)
        return o, conv5

In [5]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True, reshape=[])

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [6]:
#placeholder 

x = tf.placeholder(tf.float32, shape=(None, 64, 64, 1))
z = tf.placeholder(tf.float32, shape=(None, 1, 1, 100))
isTrain = tf.placeholder(dtype=tf.bool)

In [7]:
G_z = Generator(z, isTrain)

In [8]:
D_real, D_real_logits = discriminator(x, isTrain)
D_fake, D_fake_logits = discriminator(G_z, isTrain, reuse=True)

In [9]:
#Loss function
def loss_function(logits_in,labels_in):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_in,labels=labels_in))

In [10]:
D_real_loss=loss_function(D_real_logits,tf.ones_like(D_real_logits))
D_fake_loss=loss_function(D_fake_logits,tf.zeros_like(D_fake_logits))
D_loss=D_real_loss+D_fake_loss
G_loss=loss_function(D_fake_logits,tf.ones_like(D_fake_logits))

In [11]:
# trainable variables for each network
T_vars = tf.trainable_variables()
D_vars = [var for var in T_vars if var.name.startswith('discriminator')]
G_vars = [var for var in T_vars if var.name.startswith('gen')]

In [12]:
batch_size = 100
lr = 0.0002
train_epoch = 1

In [13]:
# optimizer for each network
with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
    D_optim = tf.train.AdamOptimizer(lr, beta1=0.5).minimize(D_loss, var_list=D_vars)
    G_optim = tf.train.AdamOptimizer(lr, beta1=0.5).minimize(G_loss, var_list=G_vars)

In [14]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

In [15]:
#train_set = tf.image.resize_images(mnist.train.images, [64, 64]).eval()
train_set = (mnist.train.images - 0.5) / 0.5

In [16]:
train_set.shape

(55000, 28, 28, 1)

In [17]:
root = 'MNIST_DCGAN_results/'
model = 'MNIST_DCGAN_'
if not os.path.isdir(root):
    os.mkdir(root)
if not os.path.isdir(root + 'Fixed_results'):
    os.mkdir(root + 'Fixed_results')


In [18]:
np.random.seed(int(time.time()))
print('training start!')
start_time = time.time()
samples=[]

training start!


In [18]:
for epoch in range(train_epoch):
    G_losses = []
    D_losses = []
    epoch_start_time = time.time()
    for iter in range(mnist.train.num_examples // batch_size):
        # update discriminator
        x_ = train_set[iter*batch_size:(iter+1)*batch_size]
        z_ = np.random.normal(0, 1, (batch_size, 1, 1, 100))

        loss_d_, _ = sess.run([D_loss, D_optim], {x: x_, z: z_, isTrain: True})
        D_losses.append(loss_d_)

        # update generator
        z_ = np.random.normal(0, 1, (batch_size, 1, 1, 100))
        loss_g_, _ = sess.run([G_loss, G_optim], {z: z_, x: x_, isTrain: True})
        G_losses.append(loss_g_)
        
    sample_z=np.random.uniform(0,1,size=(1,1,100))
    gen_sample=sess.run(generator(z,reuse=True),feed_dict={z:sample_z})
    samples.append(gen_sample)
    epoch_end_time = time.time()
    per_epoch_ptime = epoch_end_time - epoch_start_time
    print('[%d/%d] - ptime: %.2f loss_d: %.3f, loss_g: %.3f' % ((epoch + 1), train_epoch, per_epoch_ptime, np.mean(D_losses), np.mean(G_losses)))

KeyboardInterrupt: 